In [71]:
%load_ext autoreload
%autoreload 2

from tricount import api

data, dict = api.fetch_data()
data = api.build_data(data)
detail = api.split_data(data, dict)
# postes, result = api.concat_data(detail, dict)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

    # --------------------------------- #
    # Connection to Google Sheet        #
    # and extract Spreadsheets          #
    #  - exported tricount data         #
    #  - category dictonnary            #
    # --------------------------------- #
    
 -- data fetched
(1247, 14)
Index(['Titre', 'Montant', 'Devise', 'Taux de change',
       'Montant dans la devise du tricount (EUR)', 'Type de transaction',
       'Catégorie', 'Payé par', 'Payé par Lucie', 'Payé par Vincent',
       'Impacté à Lucie', 'Impacté à Vincent', 'Date & heure',
       'URL des images'],
      dtype='object')
 -- dict fetched 
{'Alimentation': 'Quotidien', 'Shopping': 'Achats', 'Logement': 'Achats', 'Loyer & charges': 'Quotidien', 'Divertissement': 'Extra', 'Sport': 'Loisir', 'Restaurant & bar': 'Extra', 'Investissement': 'Investissement', 'Transport': 'Quotidien', 'Week-end': 'Loisir', 'Soin de santé': 'Quotidien', 'Li

In [83]:
import pandas as pd


ValueError: too many values to unpack (expected 2)

In [87]:
from typing import Dict
import warnings
import pandas as pd
import calendar as cd
warnings.simplefilter(action='ignore', category=pd.core.common.SettingWithCopyWarning)

def split_data(data: pd.DataFrame, dict: Dict):
    print("""
    # --------------------------------- #
    # Format and split data             #
    # by date and categories            #
    # --------------------------------- #
    """)
    print(data)
    detail: Dict[str, pd.DataFrame] = {}
    years = data.index.get_level_values('Année').unique().to_list()
    for year in sorted(years, reverse=True):
        df = data.loc[year,:,:]
        months = df.index.get_level_values('Mois').unique().tolist()
        detail[f'{year} (sum)'] = df.groupby(['Poste', 'Catégorie']).sum()
        detail[f'{year} (mean)'] = detail[f'{year} (sum)'] / len(months)

        for month in sorted(months, reverse=True):
            period = f'{year} {cd.month_name[month]}'
            detail[period] = df.loc[month,:].groupby(['Poste', 'Catégorie']).sum()

    for period in detail.keys():
        for category in set(dict.keys()):
            if (dict[category], category) not in detail[period].index:
                detail[period].loc[(dict[category], category), :] = [pd.NA, pd.NA, pd.NA]

    # for period in detail.keys():
    #     detail[period] = detail[period].sort_index()

    print(" -- detail build")
    print(detail[period].shape)
    print(detail[period].columns)
    print(detail[period].index)
    print(detail[period])

    return detail

detail = split_data(data, dict)



    # --------------------------------- #
    # Format and split data             #
    # by date and categories            #
    # --------------------------------- #
    
                                             Lucie Vincent   Total
Année Mois Poste          Catégorie                               
2021  6    Achats         Livres               0.0  -51.85  -51.85
                          Logement         -115.23  -36.23 -151.46
                          Shopping          -98.99   -10.0 -108.99
           Extra          Divertissement    -21.98     0.0  -21.98
                          Restaurant & bar   -92.0  -92.12 -184.12
...                                            ...     ...     ...
2023  5    Extra          Restaurant & bar    -7.8    -7.8   -15.6
           Investissement Investissement     -30.0     0.0   -30.0
           Quotidien      Alimentation      -99.71  -89.21 -188.92
                          Chien             -51.24  -51.24 -102.48
                      

In [4]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

periods = ['2022 (mean)', '2023 January', '2023 February', '2023 March', '2023 April']
people = 'Vincent'

rows = {}
for period in periods:
    rows[period] = result[period][people]
df = pd.concat(rows, axis=1).transpose()


fig = go.Figure()
for metric in [m for m in df.columns if m in ['Revenus', 'Dépenses', 'Reste à vivre', 'Capital investi', 'Epargne']]:
    fig.add_trace(go.Scatter(x=df.index.tolist(), y=np.abs(df[metric].values.tolist()), name=metric))

fig.show()

In [5]:
df2 = df.transpose()

fig = go.Figure()
for metric in df2.columns:
    fig.add_trace(go.Bar(x=df2.index.tolist(), y=np.abs(df2[metric].values.tolist()), name=metric))

fig.show()

In [7]:
rows = {}
peoples = ['Total', 'Vincent']
periods = ['2022 (mean)', '2023 January', '2023 February', '2023 March', '2023 April']

for i, period in enumerate(periods):
    
    rows[period] = detail[period][peoples]

pd.concat(rows, axis=1).style.format("{:.0f}").highlight_null(props="color: transparent;")

In [10]:
detail

{'2023 (sum)':                      Total    Lucie   Vincent
 Catégorie                                    
 Activités           -51.00   -25.50    -25.50
 Alcool              -81.00   -40.50    -40.50
 Alimentation      -1933.32 -1082.91   -850.41
 Boulangerie        -108.40   -38.20    -70.20
 Cadeau             2154.24   728.62   1425.62
 Chien             -3299.94 -1649.97  -1649.97
 Divertissement     -471.11  -290.01   -181.10
 Espèce             4000.00  2000.00   2000.00
 Formation              NaN      NaN       NaN
 Investissement     -162.00  -162.00      0.00
 Livres                 NaN      NaN       NaN
 Logement           -627.00  -324.40   -302.60
 Loyer & charges   -4582.49 -2496.41  -2086.08
 Restaurant & bar   -401.31  -182.68   -218.63
 Salaire           22176.07  9507.24  12668.83
 Services           -207.90  -167.90    -40.00
 Shopping          -1510.38  -528.86   -981.52
 Soin de santé       -52.04   -27.02    -25.02
 Sport              -146.59   -86.59    -60.00